In [2]:
# pip install QuantLib
# pip install QuantLib
import QuantLib as ql

def ql_price_american_divs(
    spot, strike, maturity,                       # maturity: ql.Date
    call_put: str,                                 # 'C' or 'P'
    rates,                                         # float (cont. zero) or list[(ql.Date, rate)]
    vols,                                          # float (Black vol) or list[(ql.Date, vol)]
    dividends,                                     # list[(date_like, cash_amount)]  date_like may be ql.Date or (Y,M,D) or datetime.date
    valuation_date: ql.Date,
    day_count = ql.Actual365Fixed(),
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE),
    t_grid: int = 400,
    x_grid: int = 200,
    vega_eps: float = 1e-4 
):
    def _to_ql_date(d):
        if isinstance(d, ql.Date):
            return d
        # support python datetime.date
        if hasattr(d, "year") and hasattr(d, "month") and hasattr(d, "day"):
            return ql.Date(d.day, d.month, d.year)
        # support tuple/list (Y,M,D)
        if isinstance(d, (tuple, list)) and len(d) == 3:
            y, m, dd = d
            return ql.Date(dd, m, y)
        raise TypeError(f"Unsupported date type: {type(d)}")

    ql.Settings.instance().evaluationDate = valuation_date

    # --- Market objects (shared) ---
    spot_q = ql.SimpleQuote(float(spot))
    u = ql.QuoteHandle(spot_q)

    # r(t) zero curve
    if isinstance(rates, (float, int)):
        rts = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, float(rates), day_count, ql.Continuous))
    else:
        r_dates, r_vals = zip(*rates)
        rts = ql.YieldTermStructureHandle(ql.ZeroCurve(list(r_dates), list(map(float, r_vals)), day_count, calendar))

    # continuous dividend yield baseline = 0 (we use cash divs explicitly)
    dts = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, 0.0, day_count, ql.Continuous))

    # cash dividend schedule (≤ maturity)
    div_schedule = []
    for d, amt in dividends:
        dq = _to_ql_date(d)
        if dq <= maturity:
            div_schedule.append(ql.FixedDividend(float(amt), dq))

    # payoff + exercise
    opt_type = ql.Option.Call if call_put.upper().startswith('C') else ql.Option.Put
    payoff   = ql.PlainVanillaPayoff(opt_type, float(strike))
    exercise = ql.AmericanExercise(valuation_date, maturity)

    def _build_vts(vols_spec, bump=0.0):
        if isinstance(vols_spec, (float, int)):
            vq  = ql.SimpleQuote(max(1e-8, float(vols_spec) + bump))
            vts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(valuation_date, calendar, ql.QuoteHandle(vq), day_count))
            return vts, vq  # return quote so we could (in principle) reuse/bump
        else:
            v_dates, v_vals = zip(*vols_spec)
            v_vals_b        = [max(1e-8, float(v) + bump) for v in v_vals]
            vts = ql.BlackVolTermStructureHandle(ql.BlackVarianceCurve(valuation_date, list(v_dates), v_vals_b, day_count))
            return vts, None

    def _price_with_vol(bump=0.0):
        vts, _ = _build_vts(vols, bump=bump)
        process = ql.BlackScholesMertonProcess(u, dts, rts, vts)
        option  = ql.VanillaOption(payoff, exercise)
        # try PDE engine with dividends; fallback to escrow if wrapper lacks the overload
        try:
            engine = ql.FdBlackScholesVanillaEngine(process, div_schedule, t_grid, x_grid)
        except TypeError:
            # escrowed-div approximation: subtract PV(divs) from spot, then PDE w/out schedule
            pv_divs = sum(d.amount() * rts.discount(d.date()) for d in div_schedule)
            spot_q.setValue(float(spot) - pv_divs)
            process = ql.BlackScholesMertonProcess(u, dts, rts, vts)
            engine  = ql.FdBlackScholesVanillaEngine(process, t_grid, x_grid)
        option.setPricingEngine(engine)
        return option

    # Base price/greeks
    opt0  = _price_with_vol(0.0)
    out   = {"npv": opt0.NPV()}
    for g in ("delta","gamma","theta"):
        try:
            out[g] = getattr(opt0, g)()
        except RuntimeError:
            out[g] = None

    # Vega: try native; if missing, compute numerically with central diff
    try:
        out["vega"] = opt0.vega()
    except RuntimeError:
        opt_p = _price_with_vol(+vega_eps)
        opt_m = _price_with_vol(-vega_eps)
        p_p, p_m = opt_p.NPV(), opt_m.NPV()
        dP_dSigma = (p_p - p_m) / (2.0 * vega_eps)    # per 1.00 vol unit
        out["vega"] = dP_dSigma * 0.01                # per 1% vol (matches QL convention)

    # restore spot in case we escrowed it
    spot_q.setValue(float(spot))
    return out

# --- example (toy) ---
# Build some dates
today = ql.Date(25, 9, 2025)
ql.Settings.instance().evaluationDate = today
mat   = ql.Date(17, 1, 2026)  # Jan 17, 2026
# Flat 5.2% zero, flat 25% vol, two cash divs:

res = ql_price_american_divs(
    spot=190, strike=200, maturity=mat, call_put='P',
    rates=0.052, vols=0.25,
    dividends=[(ql.Date(15,11,2025), 0.24), (ql.Date(15,2,2026), 0.24)],
    valuation_date=today
)
print(res)

{'npv': 15.129219066281744, 'delta': -0.6023665607218308, 'gamma': 0.016050321927662155, 'theta': -11.393200806193596, 'vega': 0.40236784561669126}


In [41]:
from polygon import RESTClient

client = RESTClient("iASuja66NhcDdq_yQu7mhYZMzBqQdA2O", pagination=False)

dividends = []
for d in client.list_dividends(
	ticker="AAPL",
	ex_dividend_date_gte="2024-01-01",
	order="asc",
	limit=60,
	sort="ex_dividend_date",
	):
    dividends.append(d)

len(dividends)

7

In [42]:
import pandas as pd
dividends = []
for d in client.list_dividends(
	ticker="MSFT",
    ex_dividend_date_gte="2025-01-01",
	order="desc",
	limit=10,
	sort="ex_dividend_date",
	):
    dividends.append(d)

df_divs=pd.DataFrame(dividends)

df_divs


#len(dividends)

,id,cash_amount,currency,declaration_date,dividend_type,ex_dividend_date,frequency,pay_date,record_date,ticker
0,Ee77348f0a7bef6d64dcf92fc50a5bf65e2caa4ba5d0a8...,0.91,USD,2025-09-15,CD,2025-11-20,4,2025-12-11,2025-11-20,MSFT
1,Eb7f1a1d0f092274a19e0db54c6af542355d01c509a1e4...,0.83,USD,2025-06-10,CD,2025-08-21,4,2025-09-11,2025-08-21,MSFT
2,Ec79a97079e6a62876fdb04c4c79b9bb34931fe1b36884...,0.83,USD,2025-03-11,CD,2025-05-15,4,2025-06-12,2025-05-15,MSFT
3,Ecef5e6c1102feab369ba58fd3711d50321893d7fd7aab...,0.83,USD,2024-12-03,CD,2025-02-20,4,2025-03-13,2025-02-20,MSFT


In [62]:
import pandas as pd
from datetime import date
from pandas.tseries.offsets import BDay

def _roll_to_business_day(ts):
    ts = pd.Timestamp(ts)
    while ts.weekday() >= 5:  # Sat/Sun -> next business day
        ts = ts + pd.Timedelta(days=1)
    return ts

def _fetch_div_history(client, ticker, start=None, end=None, limit=1000):
    """
    Pulls dividend history from Polygon. Uses auto-pagination to get everything in range.
    """
    kwargs = dict(ticker=ticker, order="asc", sort="ex_dividend_date", limit=limit)
    if start:
        kwargs["ex_dividend_date_gte"] = start
    if end:
        kwargs["ex_dividend_date_lte"] = end
    rows = [d for d in client.list_dividends(**kwargs)]
    return pd.DataFrame(rows) if rows else pd.DataFrame(columns=[
        "id","cash_amount","currency","declaration_date","dividend_type",
        "ex_dividend_date","frequency","pay_date","record_date","ticker"
    ])

def forecast_dividends(
    client,
    ticker,
    years_ahead=3,
    increase_pct=0.05,
    ref_year=None,
    date_field="ex_dividend_date",   # or "pay_date"
    use_business_day_roll=True,
    df_history=None,                 # optional: pass your own history to avoid refetch
):
    """
    Build a future dividend forecast.

    Rules:
      - Use last year's *date pattern* (by `date_field`) as the template.
      - Start from the *latest known* cash_amount and carry it forward.
      - If last year had an *increase date* (first in-year jump), apply the
        annual increase on that same mm-dd every future year.
      - Otherwise, apply the first increase on the *first date of year 2*.
    """
    if df_history is None:
        # Pull a couple years so we (a) have last year's pattern and (b) get the latest amount.
        this_year = date.today().year
        start = f"{(this_year-2)}-01-01"
        df = _fetch_div_history(client, ticker, start=start)
    else:
        df = df_history.copy()

    if df.empty:
        raise ValueError("No dividend history available to build a forecast.")

    # Normalize types
    for col in ["ex_dividend_date", "pay_date", "declaration_date", "record_date"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")
    df = df.sort_values("ex_dividend_date").reset_index(drop=True)

    # Choose reference year = last full calendar year by default
    if ref_year is None:
        today = pd.Timestamp.today().tz_localize(None)
        ref_year = today.year - 1

    if date_field not in df.columns:
        raise ValueError(f"`date_field='{date_field}'` not in dataframe columns: {df.columns.tolist()}")

    # Template dates from the reference year
    df_ref = df[df[date_field].dt.year == ref_year].dropna(subset=[date_field]).copy()
    if df_ref.empty:
        raise ValueError(f"No {date_field} rows found in reference year {ref_year}.")

    # Use the mm-dd pattern from ref_year (preserves variable days like 15/20/21, etc.)
    template_mmdd = df_ref[date_field].dt.strftime("%m-%d").tolist()
    template_mmdd = sorted(template_mmdd)  # keep calendar order
    if len(template_mmdd) < 2:
        raise ValueError("Reference year has too few dividends to infer a schedule.")

    # Detect the increase anchor inside the reference year:
    # the *first* in-year jump in cash_amount vs the immediately prior dividend
    df_all = df.sort_values(date_field if date_field in df.columns else "ex_dividend_date").reset_index(drop=True)
    diffs = df_all["cash_amount"].diff()
    in_ref = df_all[date_field].dt.year == ref_year
    inc_rows = df_all[in_ref & (diffs > 0)]
    increase_anchor = None
    if not inc_rows.empty:
        inc_dt = inc_rows.iloc[0][date_field]
        increase_anchor = (inc_dt.month, inc_dt.day)

    # Latest known amount becomes the starting level
    latest_row = df_all.iloc[-1]
    current_amount = float(latest_row["cash_amount"])
    last_known_year = int(df_all[date_field].max().year)

    # Build forecast
    out = []
    for i in range(1, years_ahead + 1):
        year = last_known_year + i
        for mmdd in template_mmdd:
            month, day = map(int, mmdd.split("-"))
            dt = pd.Timestamp(year=year, month=month, day=day)

            if use_business_day_roll:
                dt = _roll_to_business_day(dt)

            # Decide whether this date is the annual increase date
            inc_flag = False
            if increase_anchor is not None and (month, day) == increase_anchor:
                current_amount = current_amount + increase_dollars
                inc_flag = True
            elif increase_anchor is None and i >= 2 and mmdd == template_mmdd[0]:
                # No anchor last year: raise on the first date of year 2 and every year thereafter on that same first date
                current_amount = current_amount + increase_dollars
                inc_flag = True

            out.append({
                "ticker": ticker,
                date_field: dt,
                "cash_amount": float(round(current_amount, 4)),
                "increase_applied": inc_flag,
                "source": "forecast"
            })

    forecast_df = pd.DataFrame(out).sort_values(date_field).reset_index(drop=True)
    return forecast_df

In [49]:
fc = forecast_dividends(
    client,
    "MSFT",
    years_ahead=3,          # project 3 years
    increase_pct=0.08,      # e.g., 8% yearly increase
    date_field="ex_dividend_date"   # or "pay_date" if you want to anchor on pay dates
)

# Combine with your actuals if you want one table:
fc

,ticker,ex_dividend_date,cash_amount,increase_applied,source
0,MSFT,2026-02-16,0.9100,False,forecast
1,MSFT,2026-05-15,0.9100,False,forecast
2,MSFT,2026-08-17,0.9100,False,forecast
3,MSFT,2026-11-23,0.9828,True,forecast
4,MSFT,2027-02-15,0.9828,False,forecast
5,MSFT,2027-05-17,0.9828,False,forecast
6,MSFT,2027-08-16,0.9828,False,forecast
7,MSFT,2027-11-22,1.0614,True,forecast
8,MSFT,2028-02-14,1.0614,False,forecast
9,MSFT,2028-05-15,1.0614,False,forecast


In [45]:
all_divs

,id,cash_amount,currency,declaration_date,dividend_type,ex_dividend_date,frequency,pay_date,record_date,ticker,increase_applied,source
0,Ee77348f0a7bef6d64dcf92fc50a5bf65e2caa4ba5d0a8...,0.9100,USD,2025-09-15,CD,2025-11-20,4.0,2025-12-11,2025-11-20,MSFT,NaN,NaN
1,Eb7f1a1d0f092274a19e0db54c6af542355d01c509a1e4...,0.8300,USD,2025-06-10,CD,2025-08-21,4.0,2025-09-11,2025-08-21,MSFT,NaN,NaN
2,Ec79a97079e6a62876fdb04c4c79b9bb34931fe1b36884...,0.8300,USD,2025-03-11,CD,2025-05-15,4.0,2025-06-12,2025-05-15,MSFT,NaN,NaN
3,Ecef5e6c1102feab369ba58fd3711d50321893d7fd7aab...,0.8300,USD,2024-12-03,CD,2025-02-20,4.0,2025-03-13,2025-02-20,MSFT,NaN,NaN
4,NaN,0.9100,NaN,NaN,NaN,2026-02-16 00:00:00,NaN,NaN,NaN,MSFT,False,forecast
5,NaN,0.9100,NaN,NaN,NaN,2026-05-15 00:00:00,NaN,NaN,NaN,MSFT,False,forecast
6,NaN,0.9100,NaN,NaN,NaN,2026-08-17 00:00:00,NaN,NaN,NaN,MSFT,False,forecast
7,NaN,0.9828,NaN,NaN,NaN,2026-11-23 00:00:00,NaN,NaN,NaN,MSFT,True,forecast
8,NaN,0.9828,NaN,NaN,NaN,2027-02-15 00:00:00,NaN,NaN,NaN,MSFT,False,forecast
9,NaN,0.9828,NaN,NaN,NaN,2027-05-17 00:00:00,NaN,NaN,NaN,MSFT,False,forecast


In [37]:
import numpy as np


def _as_callable(x):
    return x if callable(x) else (lambda t: float(x))

def _shift_divs(dividends, dt):
    """Shift all ex-div times earlier by dt; drop any that have passed."""
    if not dividends: return []
    return [(t - dt, a) for (t, a) in dividends if (t - dt) > 0.0]

def _delta_gamma_from_grid(S_grid, V_grid, S0, h=None):
    """Central differences using interpolation on the solved grid."""
    S = np.asarray(S_grid); V = np.asarray(V_grid)
    dS = S[1] - S[0]
    if h is None:
        h = max(dS, 1e-4 * max(S0, 1.0))  # at least one grid step, or 1bp of spot
    S_minus = max(S[0], S0 - h)
    S_plus  = min(S[-1], S0 + h)
    V0      = np.interp(S0, S, V)
    V_minus = np.interp(S_minus, S, V)
    V_plus  = np.interp(S_plus,  S, V)
    delta = (V_plus - V_minus) / (S_plus - S_minus)
    # Use actual h for gamma; avoid divide-by-0 if we got clamped
    h_eff = max(1e-12, 0.5 * (S_plus - S_minus))
    gamma = (V_plus - 2.0 * V0 + V_minus) / (h_eff ** 2)
    return float(delta), float(gamma), float(V0)

def american_pde_price_and_greeks(
    S0, K, T, sigma,
    r_func, q_func=lambda t: 0.0,
    dividends=None,                 # [(t_years, cash), ...]
    cp='C',
    NS=400, NT=1200, theta=0.5, Smax=None,
    # bump sizes
    dt_years=1/365.0,               # ~1 day for theta
    dvol=1e-4,                      # 1 bp of vol for vega
    dr=1e-4,                        # 1 bp rate for rho
):
    """Returns dict with price, delta, gamma, theta (per year), vega (per 1% vol), rho (per 1%)."""
    r_func = _as_callable(r_func)
    q_func = _as_callable(q_func)
    divs   = dividends or []

    # --- base run ---
    p0, dbg0 = american_pde_price(S0, K, T, sigma, r_func, q_func, dividends=divs,
                                  cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    Sg, Vg = dbg0["S_grid"], dbg0["V0"]
    delta, gamma, v0 = _delta_gamma_from_grid(Sg, Vg, S0)

    # --- theta: move time forward by dt (shorter T, earlier dividends) ---
    dt = min(dt_years, max(1e-6, 0.5*T))  # ensure T - dt > 0
    p_short, _ = american_pde_price(S0, K, T - dt, sigma, r_func, q_func,
                                    dividends=_shift_divs(divs, dt),
                                    cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    theta_pa = (p_short - p0) / dt              # per year
    # (per day if you want: theta_pd = theta_pa / 365.0)

    # --- vega: bump σ ---
    p_up, _   = american_pde_price(S0, K, T, sigma + dvol, r_func, q_func,
                                   dividends=divs, cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    p_dn, _   = american_pde_price(S0, K, T, sigma - dvol, r_func, q_func,
                                   dividends=divs, cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    dP_dSigma = (p_up - p_dn) / (2.0 * dvol)    # per 1.00 vol unit
    vega_1pct = dP_dSigma * 0.01                # per +1% absolute vol

    # --- rho: bump r(t) everywhere by dr ---
    r_up = (lambda t: r_func(t) + dr)
    r_dn = (lambda t: r_func(t) - dr)
    p_ru, _ = american_pde_price(S0, K, T, sigma, r_up, q_func,
                                 dividends=divs, cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    p_rd, _ = american_pde_price(S0, K, T, sigma, r_dn, q_func,
                                 dividends=divs, cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    dP_dr   = (p_ru - p_rd) / (2.0 * dr)        # per 1.00 (100%) rate
    rho_1pct = dP_dr * 0.01                     # per +1% rate

    return {
        "price": float(p0),
        "delta": float(delta),
        "gamma": float(gamma),
        "theta_pa": float(theta_pa),            # per year
        "theta_pd": float(theta_pa / 365.0),    # per day (ACT/365)
        "vega_1pct": float(vega_1pct),          # per +1 vol point
        "rho_1pct": float(rho_1pct),            # per +1% rate
        "debug": dbg0
    }



def _thomas_tridiag(a, b, c, d):
    """Solve tridiagonal system with diagonals (a,b,c) and RHS d. All are 1D arrays of length n (a[0] and c[-1] ignored)."""
    n = len(d)
    cp = np.zeros(n); dp = np.zeros(n)
    bp = b.copy()
    cp[0] = c[0] / bp[0]
    dp[0] = d[0] / bp[0]
    for i in range(1, n):
        denom = bp[i] - a[i] * cp[i-1]
        cp[i] = c[i] / denom if i < n - 1 else 0.0
        dp[i] = (d[i] - a[i] * dp[i-1]) / denom
    x = np.zeros(n)
    x[-1] = dp[-1]
    for i in range(n-2, -1, -1):
        x[i] = dp[i] - cp[i] * x[i+1]
    return x

def _apply_dividend_jump(S, V_after, D):
    """Jump condition at ex-div: V_before(S) = V_after(S - D), interpolated; clamp at S=0."""
    S_shift = np.clip(S - D, 0.0, S[-1])
    return np.interp(S_shift, S, V_after)

def american_pde_price(
    S0, K, T, sigma,
    r_func,                         # callable: r(t) continuous comp
    q_func=lambda t: 0.0,           # callable: q(t) continuous comp
    dividends=None,                 # list of (t, cash_amount), 0 < t < T
    cp='C',
    NS=400, NT=1200, theta=0.5,     # theta=0.5 -> Crank–Nicolson
    Smax=None
):
    """
    Returns (price, debug_dict). Times in years (ACT/365-ish), rates are continuous comp.
    """
    dividends = sorted(dividends or [], key=lambda x: x[0])
    # Grid in S
    if Smax is None:
        D_sum = sum(d for _, d in dividends)
        base = max(S0, K)
        Smax = max(4*base, 2*base + 4*D_sum)
    S = np.linspace(0.0, Smax, NS+1)
    dS = S[1] - S[0]

    # Payoff at maturity
    if cp.upper().startswith('C'):
        payoff = np.maximum(S - K, 0.0)
    else:
        payoff = np.maximum(K - S, 0.0)

    V = payoff.copy()

    # Build time grid including dividend times
    times = list(np.linspace(0.0, T, NT+1))
    for t, _ in dividends:
        if 0 < t < T:
            times.append(t)
    times = np.array(sorted(set(times)))  # ascending
    # Map: dividend time -> amount (if multiple on same day, they sum)
    div_map = {}
    for t, d in dividends:
        div_map[t] = div_map.get(t, 0.0) + d
    div_times = set(div_map.keys())

    # Backward in time
    for j in range(len(times)-1, 0, -1):
        t_cur  = times[j]
        t_prev = times[j-1]
        dt = t_cur - t_prev
        t_mid = t_prev + theta*dt  # time where coefficients are sampled

        r = float(r_func(t_mid))
        q = float(q_func(t_mid))
        sig2 = float(sigma)**2

        # Coeffs for interior nodes 1..NS-1
        Si   = S[1:-1]
        A = 0.5 * sig2 * (Si**2) / (dS**2)
        B = (r - q) * Si / (2.0*dS)

        # LHS (theta part)
        lower = -theta * dt * (A - B)
        diag  = 1.0 + theta * dt * (2.0*A + r)
        upper = -theta * dt * (A + B)

        # RHS ((1-theta) part)
        lower_r = (1.0 - theta) * dt * (A - B)
        diag_r  = 1.0 - (1.0 - theta) * dt * (2.0*A + r)
        upper_r = (1.0 - theta) * dt * (A + B)

        # Boundary values at t_prev (simple, practical)
        # For better accuracy you can use DF-based boundaries; these work well with a large enough Smax.
        if cp.upper().startswith('C'):
            V0   = 0.0
            VN   = S[-1]  # call ~ linear in S at far boundary
        else:
            V0   = K      # American put: can exercise to get K at S=0
            VN   = 0.0

        rhs = lower_r * V[:-2] + diag_r * V[1:-1] + upper_r * V[2:]
        rhs[0]  -= lower[0] * V0
        rhs[-1] -= upper[-1] * VN

        # Solve tridiagonal
        V_inner = _thomas_tridiag(
            a=np.r_[0.0, lower],             # pad to length
            b=diag,
            c=np.r_[upper, 0.0],
            d=rhs
        )

        V = np.empty_like(V)
        V[0]  = V0
        V[-1] = VN
        V[1:-1] = V_inner

        # American early exercise
        if cp.upper().startswith('C'):
            V = np.maximum(V, S - K)
        else:
            V = np.maximum(V, K - S)

        # If we just landed on an ex-div date, apply the jump to go to t^- and re-enforce early exercise
        if t_prev in div_times:
            D = div_map[t_prev]
            V = _apply_dividend_jump(S, V, D)
            # early exercise right before ex-div (important for calls with divs)
            if cp.upper().startswith('C'):
                V = np.maximum(V, S - K)
            else:
                V = np.maximum(V, K - S)

    # Interpolate back to S0
    price = np.interp(S0, S, V)
    debug = {"S_grid": S, "V0": V, "times": times}
    return price, debug

# --- example usage ---
# r_func and q_func can come from your saved curve; here are simple flats:
r_func = lambda t: 0.052      # 5.2% cont
q_func = lambda t: 0.00       # continuous yield baseline (keep 0 if using discrete divs)

divs = [(0.15, 0.24), (0.45, 0.24)]  # (time in years, cash)
p, dbg = american_pde_price(
    S0=190, K=200, T=1.33, sigma=0.25,
    r_func=r_func, q_func=q_func, dividends=divs,
    cp='P', NS=400, NT=1200
)
print("American put (PDE) ≈", round(p, 4))
for i in range(10):
    greeks = american_pde_price_and_greeks(
        S0=190, K=200, T=1.33, sigma=0.25,
        r_func=r_func, q_func=q_func, dividends=divs,
        cp='P', NS=400, NT=1200,
        dt_years=1/365, dvol=1e-4, dr=1e-4
    )

#print(greeks)
# -> {price, delta, gamma, theta_pa, theta_pd, vega_1pct, rho_1pct, ...}

American put (PDE) ≈ 11.312


In [42]:
# pip install scipy
import numpy as np
from scipy.linalg import solve_banded


def american_pde_price_banded(
    S0, K, T, sigma,
    r_func, q_func=lambda t: 0.0,
    dividends=None,                 # [(t_years, cash), ...]
    cp='C',
    NS=400, NT=800,                 # you can often use NT ~ NS with Rannacher
    theta=0.5,                      # 0.5 = Crank–Nicolson
    rannacher=2,                    # first 2 steps implicit Euler to smooth payoff kink
    Smax=None,
    return_grid=False
):
    """FDM PDE with discrete cash dividends, early exercise, banded solver."""
    dividends = sorted(dividends or [], key=lambda x: x[0])

    # spot grid
    if Smax is None:
        D_sum = sum(d for _, d in dividends)
        base = max(S0, K)
        Smax = max(4*base, 2*base + 4*D_sum)
    S = np.linspace(0.0, Smax, NS+1).astype(np.float64)
    dS = S[1] - S[0]

    # payoff at maturity
    V = (S - K).clip(min=0.0) if cp.upper().startswith('C') else (K - S).clip(min=0.0)

    # time grid (include dividend times exactly)
    times = list(np.linspace(0.0, T, NT+1))
    for t, _ in dividends:
        if 0 < t < T: times.append(t)
    times = np.array(sorted(set(times)), dtype=np.float64)  # ascending
    div_amt = {t: 0.0 for t, _ in dividends if 0 < t <= T}
    for t, a in dividends:
        if 0 < t <= T: div_amt[t] = div_amt.get(t, 0.0) + float(a)
    div_times = set(div_amt.keys())

    # optional full grid storage
    V_grid = None
    if return_grid:
        V_grid = np.empty((len(times), NS+1), dtype=np.float64)
        V_grid[-1] = V

    # prealloc work arrays
    N = NS - 1                         # number of interior nodes
    Si = S[1:-1]
    lower = np.empty(N, dtype=np.float64)
    diag  = np.empty(N, dtype=np.float64)
    upper = np.empty(N, dtype=np.float64)
    rhs   = np.empty(N, dtype=np.float64)
    # banded matrix container (3, N): [upper; diag; lower]
    ab = np.zeros((3, N), dtype=np.float64)

    # backward time stepping
    steps_done = 0
    for j in range(len(times)-1, 0, -1):
        t_cur, t_prev = times[j], times[j-1]
        dt = t_cur - t_prev
        th = 1.0 if steps_done < rannacher else theta
        t_mid = t_prev + th*dt

        r = float(r_func(t_mid)); q = float(q_func(t_mid))
        sig2 = float(sigma)*float(sigma)

        A = 0.5 * sig2 * (Si**2) / (dS**2)
        B = (r - q) * Si / (2.0*dS)

        # LHS (theta part)
        lower[:] = -th * dt * (A - B)   # length N
        diag[:]  =  1.0 + th * dt * (2.0*A + r)
        upper[:] = -th * dt * (A + B)

        # RHS ((1-theta) part)
        lower_r = (1.0 - th) * dt * (A - B)
        diag_r  = 1.0 - (1.0 - th) * dt * (2.0*A + r)
        upper_r = (1.0 - th) * dt * (A + B)

        # boundaries
        if cp.upper().startswith('C'):
            V0, VN = 0.0, S[-1]
        else:
            V0, VN = K, 0.0

        rhs[:] = lower_r * V[:-2] + diag_r * V[1:-1] + upper_r * V[2:]
        rhs[0]  -= lower[0] * V0
        rhs[-1] -= upper[-1] * VN

        # build banded tri-diagonal:
        # ab[0,1:] = upper[:-1], ab[1,:] = diag, ab[2,:-1] = lower[1:]
        ab[0, 1:] = upper[:-1]
        ab[1, :]  = diag
        ab[2, :-1]= lower[1:]
        # solve
        V_inner = solve_banded((1, 1), ab, rhs, overwrite_ab=True, overwrite_b=True, check_finite=False)

        V[0], V[-1] = V0, VN
        V[1:-1] = V_inner

        # early exercise
        if cp.upper().startswith('C'):
            np.maximum(V, S - K, out=V)
        else:
            np.maximum(V, K - S, out=V)

        # discrete dividend jump at t_prev
        if t_prev in div_times:
            D = div_amt[t_prev]
            S_shift = np.clip(S - D, 0.0, S[-1])
            V[:] = np.interp(S_shift, S, V)
            # re-enforce exercise right before ex-div
            if cp.upper().startswith('C'):
                np.maximum(V, S - K, out=V)
            else:
                np.maximum(V, K - S, out=V)

        if return_grid:
            V_grid[j-1] = V

        steps_done += 1

    price = float(np.interp(S0, S, V))
    debug = {"S_grid": S, "times": times}
    if return_grid:
        debug["V_grid"] = V_grid
    else:
        debug["V0"] = V
    return price, debug


def american_pde_price_and_greeks_banded(
    S0, K, T, sigma,
    r_func, q_func=lambda t: 0.0,
    dividends=None,                 # [(t_years, cash), ...]
    cp='C',
    NS=400, NT=1200, theta=0.5, Smax=None,
    # bump sizes
    dt_years=1/365.0,               # ~1 day for theta
    dvol=1e-4,                      # 1 bp of vol for vega
    dr=1e-4,                        # 1 bp rate for rho
):
    """Returns dict with price, delta, gamma, theta (per year), vega (per 1% vol), rho (per 1%)."""
    r_func = _as_callable(r_func)
    q_func = _as_callable(q_func)
    divs   = dividends or []

    # --- base run ---
    p0, dbg0 = american_pde_price_banded(S0, K, T, sigma, r_func, q_func, dividends=divs,
                                  cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    Sg, Vg = dbg0["S_grid"], dbg0["V0"]
    delta, gamma, v0 = _delta_gamma_from_grid(Sg, Vg, S0)

    # --- theta: move time forward by dt (shorter T, earlier dividends) ---
    dt = min(dt_years, max(1e-6, 0.5*T))  # ensure T - dt > 0
    p_short, _ = american_pde_price_banded(S0, K, T - dt, sigma, r_func, q_func,
                                    dividends=_shift_divs(divs, dt),
                                    cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    theta_pa = (p_short - p0) / dt              # per year
    # (per day if you want: theta_pd = theta_pa / 365.0)

    # --- vega: bump σ ---
    p_up, _   = american_pde_price_banded(S0, K, T, sigma + dvol, r_func, q_func,
                                   dividends=divs, cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    p_dn, _   = american_pde_price_banded(S0, K, T, sigma - dvol, r_func, q_func,
                                   dividends=divs, cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    dP_dSigma = (p_up - p_dn) / (2.0 * dvol)    # per 1.00 vol unit
    vega_1pct = dP_dSigma * 0.01                # per +1% absolute vol

    # --- rho: bump r(t) everywhere by dr ---
    r_up = (lambda t: r_func(t) + dr)
    r_dn = (lambda t: r_func(t) - dr)
    p_ru, _ = american_pde_price_banded(S0, K, T, sigma, r_up, q_func,
                                 dividends=divs, cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    p_rd, _ = american_pde_price_banded(S0, K, T, sigma, r_dn, q_func,
                                 dividends=divs, cp=cp, NS=NS, NT=NT, theta=theta, Smax=Smax)
    dP_dr   = (p_ru - p_rd) / (2.0 * dr)        # per 1.00 (100%) rate
    rho_1pct = dP_dr * 0.01                     # per +1% rate

    return {
        "price": float(p0),
        "delta": float(delta),
        "gamma": float(gamma),
        "theta_pa": float(theta_pa),            # per year
        "theta_pd": float(theta_pa / 365.0),    # per day (ACT/365)
        "vega_1pct": float(vega_1pct),          # per +1 vol point
        "rho_1pct": float(rho_1pct),            # per +1% rate
        "debug": dbg0
    }


for i in range(100):
    greeks = american_pde_price_and_greeks_banded(
        S0=190, K=200, T=1.33, sigma=0.25,
        r_func=r_func, q_func=q_func, dividends=divs,
        cp='P', NS=400, NT=1200,
        dt_years=1/365, dvol=1e-4, dr=1e-4
    )

In [37]:
# pip install QuantLib
import QuantLib as ql
import math

def zero_curve_from_csv(valuation_date: ql.Date, rows, day_count=ql.Actual365Fixed(), cal=ql.UnitedStates(ql.UnitedStates.NYSE)):
    """
    rows: iterable of (T_years, r_cont_zero). Builds a ZeroCurve dated from valuation_date.
    """
    dates = [valuation_date] + [valuation_date + ql.Period(int(round(T*365)), ql.Days) for T, _ in rows]
    rates = [rows[0][1]] + [float(r) for _, r in rows]  # pad front; QL expects >= 2 nodes
    return ql.YieldTermStructureHandle(ql.ZeroCurve(dates, rates, day_count, cal))

class QLAmericanPricer:
    def __init__(self, valuation_date: ql.Date, r_curve: ql.YieldTermStructureHandle,
                 init_vol: float = 0.20, cal=ql.UnitedStates(ql.UnitedStates.NYSE), dc=ql.Actual365Fixed(),
                 t_grid: int = 400, x_grid: int = 200):
        ql.Settings.instance().evaluationDate = valuation_date
        self.val_date = valuation_date
        self.cal, self.dc = cal, dc
        self.t_grid, self.x_grid = t_grid, x_grid

        # live quotes you can bump
        self.spot_q = ql.SimpleQuote(0.0)
        self.vol_q  = ql.SimpleQuote(max(1e-8, init_vol))

        # handles
        self.u   = ql.QuoteHandle(self.spot_q)
        self.vts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(valuation_date, cal, ql.QuoteHandle(self.vol_q), dc))
        self.rts = r_curve
        self.dts = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, 0.0, dc, ql.Continuous))  # cont. q=0; use cash divs explicitly

        # stochastic process reused across prices
        self.process = ql.BlackScholesMertonProcess(self.u, self.dts, self.rts, self.vts)

    def _div_schedule(self, dividends):
        sched = []
        if not dividends: return sched
        for d, amt in dividends:
            dq = d if isinstance(d, ql.Date) else ql.Date(d.day, d.month, d.year)
            if dq > self.val_date:
                sched.append(ql.FixedDividend(float(amt), dq))
        return sched

    def price(self, S, K, maturity: ql.Date, cp: str,
              vol: float | None = None, dividends: list[tuple] | None = None,
              want_greeks: bool = True, vega_eps: float = 1e-4):
        # set live quotes
        self.spot_q.setValue(float(S))
        if vol is not None:
            self.vol_q.setValue(max(1e-8, float(vol)))

        payoff   = ql.PlainVanillaPayoff(ql.Option.Call if cp.upper().startswith('C') else ql.Option.Put, float(K))
        exercise = ql.AmericanExercise(self.val_date, maturity)
        option   = ql.VanillaOption(payoff, exercise)

        divs = self._div_schedule(dividends or [])
        # try FDM engine with explicit dividend schedule
        engine = None
        try:
            engine = ql.FdBlackScholesVanillaEngine(self.process, divs, self.t_grid, self.x_grid)
        except TypeError:
            # fallback: escrowed dividend approx (subtract PV of cash divs from spot)
            pv = sum(d.amount() * self.rts.discount(d.date()) for d in divs)
            self.spot_q.setValue(float(S) - pv)
            engine = ql.FdBlackScholesVanillaEngine(self.process, self.t_grid, self.x_grid)

        option.setPricingEngine(engine)

        out = {"theo": option.NPV()}
        if not want_greeks:
            # restore spot if we escrowed it
            self.spot_q.setValue(float(S))
            return out

        # Greks: ask engine; if missing, do light numeric where needed
        for g in ("delta","gamma","theta"):
            try:
                out[g] = getattr(option, g)()
            except RuntimeError:
                out[g] = None

        # vega: some PDE engines don't provide it → numeric
        try:
            out["vega"] = option.vega()
        except RuntimeError:
            v0 = self.vol_q.value()
            self.vol_q.setValue(v0 + vega_eps); p_up = ql.VanillaOption(payoff, exercise); p_up.setPricingEngine(engine); upv = p_up.NPV()
            self.vol_q.setValue(v0 - vega_eps); p_dn = ql.VanillaOption(payoff, exercise); p_dn.setPricingEngine(engine); dnv = p_dn.NPV()
            self.vol_q.setValue(v0)
            out["vega"] = ((upv - dnv) / (2*vega_eps)) * 0.01  # per 1 vol point

        # restore spot if escrow path
        self.spot_q.setValue(float(S))
        return out

In [38]:
# 1) Build a zero curve once from your saved CSV (T, r) rows:
val = ql.Date(25, 9, 2025)
rows = [(0.25, 0.0530), (0.50, 0.0538), (1.00, 0.0542), (2.00, 0.0520), (5.00, 0.0490)]
r_curve = zero_curve_from_csv(val, rows)
pricer  = QLAmericanPricer(val, r_curve, init_vol=0.25)

# 2) Price legs quickly by bumping spot/vol only:
mat = ql.Date(17, 1, 2026)
divs = [(ql.Date(15,11,2025), 0.24)]  # cash dividends ≤ maturity
print(pricer.price(S=190, K=200, maturity=mat, cp='P', vol=0.25, dividends=divs))
# {'theo': ..., 'delta': ..., 'gamma': ..., 'theta': ..., 'vega': ...}

# 3) When the user edits vol, just call pricer.price(..., vol=new_sigma). No rebuilds.

{'theo': 15.100537259378733, 'delta': -0.6022422429340001, 'gamma': 0.016101190493306924, 'theta': -11.323617382807054, 'vega': 0.40219571363566686}


In [63]:
import pandas as pd
from datetime import date
from pandas.tseries.offsets import DateOffset

ALLOWED_FREQ_STEPS = {1:12, 2:6, 3:4, 4:3, 6:2, 12:1}

def _roll_to_business_day(ts):
    ts = pd.Timestamp(ts)
    while ts.weekday() >= 5:  # Sat/Sun -> next business day
        ts = ts + pd.Timedelta(days=1)
    return ts

def _fetch_div_history(client, ticker, start=None, end=None, limit=1000):
    kwargs = dict(ticker=ticker, order="asc", sort="ex_dividend_date", limit=limit)
    if start:
        kwargs["ex_dividend_date_gte"] = start
    if end:
        kwargs["ex_dividend_date_lte"] = end
    rows = [d for d in client.list_dividends(**kwargs)]
    cols = ["id","cash_amount","currency","declaration_date","dividend_type",
            "ex_dividend_date","frequency","pay_date","record_date","ticker"]
    return pd.DataFrame(rows) if rows else pd.DataFrame(columns=cols)

def _parse_mmdd_list(dates_like, base_year):
    """Accepts strings 'YYYY-MM-DD' or 'MM-DD', or pandas/py dates; returns sorted ['MM-DD', ...]."""
    out = []
    for x in dates_like:
        if isinstance(x, (pd.Timestamp, )):
            ts = x
        else:
            s = str(x)
            if len(s) == 5 and s[2] == "-":  # MM-DD
                ts = pd.to_datetime(f"{base_year}-{s}", errors="raise")
            else:  # try full date
                ts = pd.to_datetime(s, errors="raise")
        out.append(ts.strftime("%m-%d"))
    return sorted(out)

def _generate_template_from_first(mmdd_first, freq, base_year):
    step = ALLOWED_FREQ_STEPS.get(freq)
    if step is None:
        raise ValueError(f"Unsupported frequency {freq}. Use one of {sorted(ALLOWED_FREQ_STEPS)}.")
    start = pd.Timestamp(f"{base_year}-{mmdd_first}")
    res = []
    for k in range(freq):
        dt = start + DateOffset(months=step*k)
        res.append(dt.strftime("%m-%d"))
    return sorted(res)

def _complete_template(mmdd_list, freq, base_year):
    """If you provided < freq dates, fill the rest by stepping equal months."""
    step = ALLOWED_FREQ_STEPS.get(freq)
    if step is None:
        raise ValueError(f"Unsupported frequency {freq}. Use one of {sorted(ALLOWED_FREQ_STEPS)}.")
    if len(mmdd_list) >= freq:
        return sorted(mmdd_list)[:freq]
    if len(mmdd_list) == 1:
        return _generate_template_from_first(mmdd_list[0], freq, base_year)
    # 2..freq-1 seeds: continue stepping from the last provided
    seeds = sorted(mmdd_list)
    cur = pd.Timestamp(f"{base_year}-{seeds[-1]}")
    out = seeds[:]
    while len(out) < freq:
        cur = cur + DateOffset(months=step)
        out.append(cur.strftime("%m-%d"))
    return sorted(out)

def forecast_dividends(
    client,
    ticker,
    years_ahead=3,
    increase_dollars=0.05,
    ref_year=None,
    date_field="ex_dividend_date",      # or "pay_date"
    use_business_day_roll=True,
    df_history=None,                    # pass your own history to avoid refetch
    frequency_override=None,            # 1,2,3,4,6,12
    first_dates_override=None           # ["MM-DD", ...] or ["YYYY-MM-DD", ...]
):
    """
    Build a future dividend forecast.

    Template dates:
      - DEFAULT: use the date pattern from `ref_year` (last full year).
      - OVERRIDE: if `first_dates_override` and/or `frequency_override` are given,
        auto-build the yearly template from those seeds.

    Increase logic:
      - If `ref_year` had an in-year *first* increase (cash_amount jumped),
        apply the annual increase on that (mapped) date each future year.
      - Else apply the increase on the first template date of the *second* forecast year.
    """
    # History
    if df_history is None:
        this_year = date.today().year
        df = _fetch_div_history(client, ticker, start=f"{this_year-3}-01-01")
    else:
        df = df_history.copy()

    if df.empty:
        raise ValueError("No dividend history available to build a forecast.")

    # Normalize types
    for col in ["ex_dividend_date", "pay_date", "declaration_date", "record_date"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")
    df = df.sort_values("ex_dividend_date").reset_index(drop=True)

    if ref_year is None:
        today = pd.Timestamp.today().tz_localize(None)
        ref_year = today.year - 1

    if date_field not in df.columns:
        raise ValueError(f"`date_field='{date_field}'` not in dataframe: {df.columns.tolist()}")

    # Base years
    last_known_year = int(df[date_field].max().year)
    first_forecast_year = last_known_year + 1

    # ---- Template (per-year schedule) ----
    template_mmdd = None

    if first_dates_override or frequency_override:
        # Build from overrides
        if first_dates_override:
            seeds = _parse_mmdd_list(first_dates_override, base_year=first_forecast_year)
        else:
            # No dates given; derive seeds from ref year’s actual pattern
            df_ref = df[df[date_field].dt.year == ref_year].dropna(subset=[date_field])
            if df_ref.empty:
                raise ValueError(f"No {date_field} rows in reference year {ref_year}.")
            seeds = sorted(df_ref[date_field].dt.strftime("%m-%d").tolist())

        if frequency_override is None:
            freq = len(seeds)
        else:
            freq = int(frequency_override)

        template_mmdd = _complete_template(seeds, freq, base_year=first_forecast_year)

    else:
        # Default: copy last full year's pattern
        df_ref = df[df[date_field].dt.year == ref_year].dropna(subset=[date_field])
        if df_ref.empty:
            raise ValueError(f"No {date_field} rows in reference year {ref_year}.")
        template_mmdd = sorted(df_ref[date_field].dt.strftime("%m-%d").tolist())
        if len(template_mmdd) < 1:
            raise ValueError("Reference year has too few dividends to infer a schedule.")

    # ---- Increase anchor detection (from history) ----
    df_all = df.sort_values(date_field).reset_index(drop=True)
    diffs = df_all["cash_amount"].diff()
    in_ref = df_all[date_field].dt.year == ref_year
    inc_rows = df_all[in_ref & (diffs > 0)]
    increase_anchor = None  # (month, day) from *history*
    if not inc_rows.empty:
        inc_dt = inc_rows.iloc[0][date_field]
        increase_anchor = (inc_dt.month, inc_dt.day)

    # Map anchor to our template (if it’s not an exact match, pick the first template date ON/AFTER it)
    mapped_anchor = None
    if increase_anchor is not None:
        anchor_mmdd = f"{increase_anchor[0]:02d}-{increase_anchor[1]:02d}"
        if anchor_mmdd in template_mmdd:
            mapped_anchor = anchor_mmdd
        else:
            # pick first >= anchor; if none, wrap to first
            later = [d for d in template_mmdd if d >= anchor_mmdd]
            mapped_anchor = later[0] if later else template_mmdd[0]

    # Latest amount → starting level
    current_amount = float(df_all.iloc[-1]["cash_amount"])

    # ---- Build forecast ----
    out = []
    for i in range(1, years_ahead + 1):
        year = last_known_year + i
        for mmdd in template_mmdd:
            month, day = map(int, mmdd.split("-"))
            dt = pd.Timestamp(year=year, month=month, day=day)
            if use_business_day_roll:
                dt = _roll_to_business_day(dt)

            inc_applied = False
            if mapped_anchor is not None and mmdd == mapped_anchor:
                current_amount = current_amount+ increase_dollars
                inc_applied = True
            elif mapped_anchor is None and i >= 2 and mmdd == template_mmdd[0]:
                # No historical anchor -> raise on first date of year 2 and onward
                current_amount = current_amount+ increase_dollars
                inc_applied = True

            out.append({
                "ticker": ticker,
                date_field: dt,
                "cash_amount": float(round(current_amount, 4)),
                "increase_applied": inc_applied,
                "source": "forecast"
            })

    forecast_df = pd.DataFrame(out).sort_values(date_field).reset_index(drop=True)
    return forecast_df

In [64]:
fc = forecast_dividends(
    client, "MSFT",
    years_ahead=3,
    increase_dollars=0.07,
    date_field="ex_dividend_date",
    frequency_override=4,
    first_dates_override=["2026-02-20"]  # seeds Q1; Q2/Q3/Q4 are auto at +3 months
)

fc

,ticker,ex_dividend_date,cash_amount,increase_applied,source
0,MSFT,2026-02-20,0.98,True,forecast
1,MSFT,2026-05-20,0.98,False,forecast
2,MSFT,2026-08-20,0.98,False,forecast
3,MSFT,2026-11-20,0.98,False,forecast
4,MSFT,2027-02-22,1.05,True,forecast
5,MSFT,2027-05-20,1.05,False,forecast
6,MSFT,2027-08-20,1.05,False,forecast
7,MSFT,2027-11-22,1.05,False,forecast
8,MSFT,2028-02-21,1.12,True,forecast
9,MSFT,2028-05-22,1.12,False,forecast


In [84]:
def convert_fc_to_ql_divs(fc):
    ql_divs=[]
    cash_amounts=fc["cash_amount"].tolist()
    ex_dates=fc["ex_dividend_date"].tolist()
    for pair in zip(ex_dates, cash_amounts):
        #print(pair)
        ql_date=ql.Date(pair[0].day, pair[0].month, pair[0].year)
        ql_divs.append((ql_date, pair[1]))

    ql_divs
    return ql_divs

print(convert_fc_to_ql_divs(fc))

df_yc=pd.read_csv("yc_snap.csv")


[(Date(20,2,2026), 0.98), (Date(20,5,2026), 0.98), (Date(20,8,2026), 0.98), (Date(20,11,2026), 0.98), (Date(22,2,2027), 1.05), (Date(20,5,2027), 1.05), (Date(20,8,2027), 1.05), (Date(22,11,2027), 1.05), (Date(21,2,2028), 1.12), (Date(22,5,2028), 1.12), (Date(21,8,2028), 1.12), (Date(20,11,2028), 1.12)]


In [ ]:
df_yc
yrs=df_yc["yrs"].tolist()
rates=df_yc["rate"].tolist()
yield_rows=list(zip(yrs, rates))
dt_today=ql.Date(25, 9, 2025)
val=ql.Date(25, 9, 2025)
ql_yield_curve=zero_curve_from_csv(val, yield_rows)





<QuantLib.QuantLib.YieldTermStructureHandle; proxy of <Swig Object of type 'Handle< YieldTermStructure > *' at 0x319b9d500> >

In [82]:
yield_rows

[(1.0, 0.0365),
 (2.0, 0.034),
 (3.0, 0.0334),
 (5.0, 0.0336),
 (7.0, 0.0345),
 (10.0, 0.0361),
 (15.0, 0.0381),
 (30.0, 0.0386),
 (nan, nan)]